In [1]:
import nltk
nltk.download('punkt')
from pymongo import MongoClient
import numpy as np
import json
import pandas as pd
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/vadim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
client = MongoClient('localhost', port=27017)
db = client['news_classification']
articles_coll = db['GameSpotArticles']
tags_coll = db['Tags']

In [ ]:
with open("./Developers.json", 'r') as f:
    Developers = json.load(f)
with open("./games.json", 'r') as f:
    games = json.load(f)
with open("./Publishers.json", 'r') as f:
    Publishers = json.load(f)

def prepare_tokens(v):
    v = v.lower()
    tokens = tuple(word_tokenize(v))
    return {"value": v, "tokens": tokens, "l": len(tokens)}

games = [{f"type": "game", **prepare_tokens(v)} for v in games]
developers = [{f"type": "company", **prepare_tokens(v)} for v in Developers]
Publishers = [{f"type": "company", **prepare_tokens(v)} for v in Publishers]
print(games)
tags = pd.DataFrame(games + developers + Publishers)
tags = tags.drop_duplicates(keep='first')
print(tags.head())
tags = tags.sort_values('l')

tags = tags.to_json(orient="records")
tags = json.loads(tags)

with open("./tags.json", 'w') as f:
    json.dump(tags, f)

tags_coll.insert_many(tags)

In [6]:
!pip3 install wikipedia

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=94dc2dd2eb72af326786e0d99b123eedf09c259b4e33d339f23a8e7486e1a7fd
  Stored in directory: /home/vadim/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [14]:
import pandas as pd
import wikipedia as wp

def pars(title):
    html = wp.page(title = title).html().encode("UTF-8")
    try:
        df = pd.read_html(html)[1] # Try 2nd table first as most pages contain contents table first
    except IndexError:
        df = pd.read_html(html)[0]
    return df

In [18]:
PC_games = pars('List of best-selling PC games')
PC_games.head()

,Game,Total copies sold,Series,Release date,Genre(s),Developer(s),Publisher(s)
0,PUBG: Battlegrounds,42 million[1],PUBG Universe,"December 20, 2017",Battle royale,PUBG Studios,Krafton
1,Minecraft,33 million[2],Minecraft,"November 18, 2011[a]","Sandbox, survival",Mojang Studios,Mojang Studios
2,Terraria,23 million[5],â,"May 16, 2011",Action-adventure,Re-Logic,Re-Logic
3,Diablo III,20 million[6][b],Diablo,"May 15, 2012",Action role-playing,Blizzard Entertainment,Blizzard Entertainment
4,Garry's Mod,20 million[7],â,"November 29, 2006",Sandbox,Facepunch Studios,Valve


In [17]:
Switch = pars("List of best-selling Nintendo Switch video games")
Switch.head()

,Rank,Title,Copies sold,As of,Release date[a],Genre(s),Developer(s),Publisher(s)
0,1,Mario Kart 8 Deluxe,53.79 million[4],"March 31, 2023","April 28, 2017",Kart racing,Nintendo EPD,Nintendo
1,2,Animal Crossing: New Horizons,42.21 million[4],"March 31, 2023","March 20, 2020",Social simulation,Nintendo EPD,Nintendo
2,3,Super Smash Bros. Ultimate,31.09 million[4],"March 31, 2023","December 7, 2018",Fighting,Bandai Namco StudiosSora Ltd.,Nintendo
3,4,The Legend of Zelda: Breath of the Wild,29.81 million[4],"March 31, 2023","March 3, 2017",Action-adventure,Nintendo EPD,Nintendo
4,5,PokÃ©mon Sword and Shield,25.82 million[4],"March 31, 2023","November 15, 2019",Role-playing,Game Freak,The PokÃ©mon CompanyNintendo


In [46]:
Xbox = pars("List of Xbox Series X and Series S games")
Xbox.head()

Title   
                        Title   
0     3 out of 10: Season One  \
1              Aeterna Noctis   
2           AEW Fight Forever   
3            Age of Wonders 4   
4  Ailment & Endurance Bundle   

                                            Genre(s)   
                                            Genre(s)   
0  .mw-parser-output .hlist dl,.mw-parser-output ...  \
1                                       Metroidvania   
2                                             Sports   
3                                Turn-based strategy   
4                                   Action-adventure   

                   Developer(s)                  Publisher(s) Release date[c]   
                   Developer(s)                  Publisher(s)              JP   
0  Terrible Posture Games, Inc.  Terrible Posture Games, Inc.             TBA  \
1         Aeternum Game Studios         Aeternum Game Studios      Unreleased   
2                        Yuke's                     AEW Games             TBA   
3               Triumph Studios           Paradox Interactive      Unreleased   
4                Ivan Panasenko                   EpiXR Games             TBA   

                              Addons Ref.  
             NA           PAL Addons Ref.  
0   Mar 3, 2021           TBA  SD OP  [2]  
1  Dec 15, 2021  Dec 15, 2021    NaN  [3]  
2           TBA           TBA    NaN  [4]  
3   May 2, 2023   May 2, 2023    NaN  [5]  
4  Feb 17, 2021           TBA  SD OP  [6]

In [54]:
Xbox = Xbox.drop(0)

In [48]:
Xbox.columns = [x[0] for x in Xbox.columns]

In [55]:
Xbox.head()

,Title,Genre(s),Developer(s),Publisher(s)
1,Aeterna Noctis,Metroidvania,Aeternum Game Studios,Aeternum Game Studios
2,AEW Fight Forever,Sports,Yuke's,AEW Games
3,Age of Wonders 4,Turn-based strategy,Triumph Studios,Paradox Interactive
4,Ailment & Endurance Bundle,Action-adventure,Ivan Panasenko,EpiXR Games
5,Alan Wake Remastered,Action-adventure,Remedy Entertainment,Epic Games Publishing


In [20]:
ps5 = pars("List of PlayStation 5 games")
ps5.head()

Title             Genre(s)   
                                     Title             Genre(s)   
0  3D Billiards: Pool & Snooker Remastered               Sports  \
1                                Abandoned      Survival horror   
2                           Aeterna Noctis         Metroidvania   
3                        AEW Fight Forever               Sports   
4                         Age of Wonders 4  Turn-based strategy   

            Developer(s)           Publisher(s) Release date                 
            Developer(s)           Publisher(s)           JP            NA   
0               Joindots               Joindots   Unreleased  Feb 19, 2021  \
1  Blue Box Game Studios  Blue Box Game Studios          TBA           TBA   
2  Aeternum Game Studios  Aeternum Game Studios   Unreleased  Dec 15, 2021   
3                 Yuke's              AEW Games          TBA           TBA   
4        Triumph Studios    Paradox Interactive   Unreleased   May 2, 2023   

                Addons Ref.  
            PAL Addons Ref.  
0  Feb 19, 2021    NaN  NaN  
1           TBA    NaN  NaN  
2  Dec 15, 2021    NaN  [3]  
3           TBA    NaN  [4]  
4   May 2, 2023    NaN  [5]

In [30]:
ps5 = ps5.drop("Release date", axis=1)
ps5.head()

/tmp/ipykernel_16010/122934322.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  ps5 = ps5.drop("Release date", axis=1)


,Title,Genre(s),Developer(s),Publisher(s),Addons,Ref.
,Title,Genre(s),Developer(s),Publisher(s),Addons,Ref.
0,3D Billiards: Pool & Snooker Remastered,Sports,Joindots,Joindots,NaN,NaN
1,Abandoned,Survival horror,Blue Box Game Studios,Blue Box Game Studios,NaN,NaN
2,Aeterna Noctis,Metroidvania,Aeternum Game Studios,Aeternum Game Studios,NaN,[3]
3,AEW Fight Forever,Sports,Yuke's,AEW Games,NaN,[4]
4,Age of Wonders 4,Turn-based strategy,Triumph Studios,Paradox Interactive,NaN,[5]
